# 从原始数据得到CLEAN_DATA的过程，去掉仅在代码段和HTML LINK中包含关键词的数据

In [1]:
import pandas as pd
import numpy as np
import random, os, re, html, time
from bs4 import BeautifulSoup

In [2]:
def remove_tag(content):
    ##remove links
    for ele in re.findall('<a href=.*?</a>', content, re.S):
        content = content.replace(ele, 'LINK')
    ##remove codes
    for ele in re.findall('<pre>.*?</pre>', content, re.S):
        content = content.replace(ele, 'CODE')
    for ele in re.findall('<code>.*?</code>', content, re.S):
        content = content.replace(ele, 'CODE')
    ##remove tags
    for ele in re.findall('</.*?>', content, re.S):
        content = content.replace(ele, '')
    for ele in re.findall('<.*?>', content, re.S):
        content = content.replace(ele, '')
    ##remove '\n'
    for ele in re.findall('\n\n', content, re.S):
        content = content.replace(ele, '\n')
    return content

In [3]:
def RemovePostNoKey(keys, df):
    newdf = pd.DataFrame()
    cnt = 0
    for index in range(0, df.shape[0]):
        notagFlag = False
        flag = False
        tagflag = True
        if (index%1000 == 0):
            print(index, cnt, newdf.shape)
        tags = df.iloc[index].Tags
        try:
            for word in keys:
                if not re.search(word, tags, re.IGNORECASE):
                    tagflag = False
        except:
            notagFlag = True
            tagflag = False
        if notagFlag:
            pass
        elif tagflag:
            flag = True
        else:
            title = df.iloc[index].Title
            titleflag = True
            for word in keys:
                if not re.search(word, title, re.IGNORECASE):
                    titleflag = False
            if not titleflag:
                body = remove_tag(df.iloc[index].Body)
                bodyflag = True
                for word in keys:
                    if not re.search(word, body, re.IGNORECASE):
                        bodyflag = False
                if bodyflag:
                    flag = True
                else:
                    pass
            else:
                flag = True
        if flag:
            d = pd.DataFrame(df.iloc[index]).T
            newdf = newdf.append([d])
            cnt += 1
    return cnt, newdf

## CSV类型文件预处理

In [13]:
file_path = '数据/original data/Amazon S3.csv'

In [14]:
#load data
df = pd.DataFrame(pd.read_csv(file_path, header=0))
print(df.shape)
df

(103010, 15)


,ViewCount,OwnerUserId,LastActivityDate,Score,Body,Tags,LastEditDate,Title,FavoriteCount,CommentCount,LastEditorUserId,AcceptedAnswerId,AnswerCount,Id,CreationDate
0,25574,797.0,2013-12-23T08:14:44.977,64,<p>I was hoping someone could help me out with...,<java><eclipse><search>,2013-12-23T08:14:44.977,Class file name must end with .class exception...,17.0,1,814074.0,6840.0,6,6816,2008-08-09T15:40:30.357
1,4761,1471.0,2013-11-26T04:44:10.260,25,"<p><a href=""http://en.wikipedia.org/wiki/Proje...",<java><concurrency><distributed>,2013-11-26T04:44:10.260,Is Project Darkstar Realistic?,6.0,0,1102512.0,485304.0,7,20034,2008-08-21T14:13:11.327
2,4271,706.0,2008-09-17T18:45:20.260,0,<p>What would be the best method for getting a...,<java><java-me><lwuit>,2008-09-08T17:20:15.487,Painting javax.microedition.lcdui.Graphics on ...,NaN,0,706.0,NaN,2,23372,2008-08-22T20:17:27.683
3,22138,3117.0,2016-07-17T22:35:02.673,33,<p>I'm using Google App Engine and Django temp...,<python><django><google-app-engine>,2016-07-17T22:35:02.673,Django templates and variable attributes,14.0,1,2291321.0,50425.0,6,35948,2008-08-30T13:20:02.740
4,65071,2148.0,2018-08-28T17:31:52.857,87,<p>Is it possible to delete an GAE application...,<google-app-engine>,2012-08-11T18:03:21.173,Deleting a Google App Engine application,11.0,0,1477076.0,1568232.0,8,42512,2008-09-03T20:32:21.263
5,1985,1647.0,2017-04-14T22:38:06.903,18,"<p>While browsing with Chrome, I noticed that ...",<google-chrome>,2010-04-12T17:39:18.187,Where is the chink in Google Chrome's armor?,3.0,5,57120.0,42912.0,13,42908,2008-09-04T00:26:55.300
6,1291,2938.0,2009-03-29T22:19:02.370,10,<p>Short version: What is the cleanest and mos...,<css><google-chrome><portability>,2008-09-06T10:19:31.473,What is the best technique for consistent form...,5.0,0,2938.0,679997.0,11,45239,2008-09-05T04:43:43.350
7,5722,4527.0,2018-05-18T09:04:09.303,2,<p>I am using a perl script to POST to Google ...,<perl><google-app-engine><https>,2008-09-09T21:13:45.557,How to get the correct Content-Length for a PO...,NaN,0,-1.0,46405.0,3,46387,2008-09-05T17:48:00.467
8,38672,366.0,2016-09-02T22:48:17.470,119,"<p>Simple one really. In SQL, if I want to se...",<google-app-engine><google-cloud-datastore><gql>,2016-09-02T22:48:17.470,Google App Engine: Is it possible to do a Gql ...,42.0,2,153407.0,47811.0,12,47786,2008-09-06T20:05:24.420
9,26376,96.0,2016-07-31T10:02:05.813,119,<p>I started an application in Google App Engi...,<python><google-app-engine>,2016-07-31T10:02:05.813,Project structure for Google App Engine,93.0,0,2666859.0,70271.0,6,48458,2008-09-07T14:08:47.233


In [15]:
keywords = ['Amazon', 'S3']
num, cdf = RemovePostNoKey(keywords, df)
print(keywords)

0 0 (0, 0)
1000 8 (8, 15)
2000 13 (13, 15)
3000 17 (17, 15)
4000 18 (18, 15)
5000 20 (20, 15)
6000 29 (29, 15)
7000 31 (31, 15)
8000 34 (34, 15)
9000 38 (38, 15)
10000 42 (42, 15)
11000 44 (44, 15)
12000 46 (46, 15)
13000 46 (46, 15)
14000 50 (50, 15)
15000 50 (50, 15)
16000 50 (50, 15)
17000 51 (51, 15)
18000 54 (54, 15)
19000 54 (54, 15)
20000 58 (58, 15)
21000 59 (59, 15)
22000 61 (61, 15)
23000 61 (61, 15)
24000 62 (62, 15)
25000 64 (64, 15)
26000 64 (64, 15)
27000 66 (66, 15)
28000 70 (70, 15)
29000 70 (70, 15)
30000 71 (71, 15)
31000 73 (73, 15)
32000 74 (74, 15)
33000 74 (74, 15)
34000 76 (76, 15)
35000 78 (78, 15)
36000 81 (81, 15)
37000 84 (84, 15)
38000 87 (87, 15)
39000 90 (90, 15)
40000 90 (90, 15)
41000 91 (91, 15)
42000 91 (91, 15)
43000 93 (93, 15)
44000 94 (94, 15)
45000 96 (96, 15)
46000 97 (97, 15)
47000 99 (99, 15)
48000 100 (100, 15)
49000 102 (102, 15)
50000 103 (103, 15)
51000 103 (103, 15)
52000 104 (104, 15)
53000 107 (107, 15)
54000 108 (108, 15)
55000 110 (110

In [16]:
cdf.to_csv('clean_amazon s3.csv', index=False)

## TXT类型文件预处理

In [4]:
def check_clean(keys, tag, title, body):
    tagflag = True
    for word in keys:
        if not re.search(word, tag, re.IGNORECASE):
            tagflag = False
    if tagflag:
        return True
    else:
        titleflag = True
        for word in keys:
            if not re.search(word, title, re.IGNORECASE):
                titleflag = False
        if not titleflag:
            bodyflag = True
            body = remove_tag(body)
            for word in keys:
                if not re.search(word, body, re.IGNORECASE):
                    bodyflag = False
            if bodyflag:
                return True
            else:
                return False
        else:
            return True

In [2]:
file_path = 'original_API_data/'
file_list = os.listdir(file_path)
file_list

['2checkout.txt',
 'admob.txt',
 'allegro.txt',
 'amazon-ec2.txt',
 'amazon-queue-service.txt',
 'amazon-s3.txt',
 'amazon-ses.txt',
 'amazon-sns.txt',
 'authorizenet.txt',
 'aws-cloud9.txt',
 'bigcommerce.txt',
 'bing-maps.txt',
 'breadcrumbs.txt',
 'brightcove.txt',
 'buffer.txt',
 'cartodb.txt',
 'coinbase.txt',
 'coupon.txt',
 'dailymotion.txt',
 'deezer.txt',
 'digitalocean.txt',
 'dropbox.txt',
 'eventbrite.txt',
 'facebook-ads.txt',
 'facebook.txt',
 'flickr.txt',
 'foursquare.txt',
 'geocoder.txt',
 'geonames.txt',
 'gmail.txt',
 'google-adsense.txt',
 'google-adwords.txt',
 'google-app-engine.txt',
 'google-checkout.txt',
 'google-cloud-dns.txt',
 'google-cloud-print.txt',
 'google-cloud-sql.txt',
 'google-directions.txt',
 'google-distance-matrix.txt',
 'google-drive.txt',
 'google-earth.txt',
 'google-fusion-tables.txt',
 'google-geocoding.txt',
 'google-maps-android.txt',
 'google-maps-places.txt',
 'google-maps.txt',
 'google-mirror.txt',
 'google-picasa.txt',
 'google-plu

In [3]:
apiname = 'youtube'
Id = []
AcceptedAnswerId = []
CreationDate = []
Score = []
ViewCount = []
Body = []
OwnerUserId = []
LastEditorUserId = []
LastEditDate = []
LastActivityDate = []
Title = []
Tags = []
AnswerCount = []
CommentCount = []
FavoriteCount = []

In [5]:
keywords = ['youtube']
cnt = 0
with open(file_path+apiname+'.txt', 'r', encoding='utf-8') as f:
    for line in f:
        cnt += 1
        if cnt%1000 == 0:
            print(cnt, len(Id))
        #print(line)
        soup = BeautifulSoup(line, 'lxml')
        row = soup.find('row')
        tag = row.get('tags')
        title = row.get('title')
        body = row.get('body')
        if check_clean(keywords, tag, title, body):
            Id.append(row.get('id'))
            Title.append(title)
            Body.append(body)
            Tags.append(tag)
            try:
                AcceptedAnswerId.append(row.get('acceptedanswerid'))
            except:
                AcceptedAnswerId.append('NULL')
            CreationDate.append(row.get('creationdate'))
            Score.append(row.get('score'))
            ViewCount.append(row.get('viewcount'))
            try:
                OwnerUserId.append(row.get('owneruserid'))
            except:
                OwnerUserId.append('NULL')
            try:
                LastEditorUserId.append(row.get('lasteditoruserid'))
                LastEditDate.append(row.get('lasteditdate'))
            except:
                LastEditorUserId.append('NULL')
                LastEditDate.append('NULL')
            try:
                LastActivityDate.append(row.get('lastactivitydate'))
            except:
                LastActivityDate.append('NULL')
            AnswerCount.append(row.get('answercount'))
            CommentCount.append(row.get('commentcount'))
            try:
                FavoriteCount.append(row.get('favoritecount'))
            except:
                FavoriteCount.append('NULL')
        else:
            pass
print(cnt, len(Id))

1000 0
2000 0
3000 0
4000 0
5000 0
6000 0
7000 0
8000 0
9000 0
10000 0
11000 0
12000 0
13000 0
14000 0
15000 0
16000 0
17000 0
18000 0
19000 0
20000 0
21000 0
22000 0
23000 0
24000 0
25000 0
26000 0
27000 0
28000 0
29000 0
30000 0
31000 0
32000 0
33000 0
34000 0
35000 0
36000 0
37000 0
38000 0
39000 0
40000 0
41000 0
42000 0
43000 0
44000 0
45000 0
46000 0
47000 0
48000 0
49000 0
50000 0
51000 0
52000 0
53000 0
54000 0
55000 0
56000 0
57000 0
58000 0
59000 0
60000 0
61000 0
62000 0
63000 0
64000 0
65000 0
66000 0
67000 0
68000 0
69000 0
70000 0
71000 0
72000 0
73000 0
74000 0
74554 0


In [11]:
towrite = {'Id':Id,'AcceptedAnswerId':AcceptedAnswerId,'CreationDate':CreationDate,'Score':Score,'ViewCount':ViewCount,'Body':Body,'OwnerUserId':OwnerUserId,'LastEditorUserId':LastEditorUserId,'LastEditDate':LastEditDate,'LastActivityDate':LastActivityDate,'Title':Title,'Tags':Tags,'AnswerCount':AnswerCount,'CommentCount':CommentCount,'FavoriteCount':FavoriteCount}
df = pd.DataFrame.from_dict(towrite, orient='index')
df.transpose().to_csv('clean_'+apiname+'.csv', index=False)

In [164]:
df = pd.DataFrame(pd.read_csv('clean_'+apiname+'.csv', header=0))
print(df.shape)
df

(4992, 15)


,CreationDate,CommentCount,LastEditorUserId,Id,AcceptedAnswerId,OwnerUserId,FavoriteCount,LastEditDate,Score,AnswerCount,Tags,Body,LastActivityDate,Title,ViewCount
0,2008-08-19T01:23:04.340,1,NaN,15486,15494.0,493.0,14.0,NaN,78,14,<c#><generics><sorting><ilist>,<p>So I came across an interesting problem tod...,2018-05-05T19:49:58.763,Sorting an IList in C#,98314
1,2008-08-19T23:09:54.720,1,NaN,17170,17295.0,1560.0,50.0,NaN,168,12,<c#><.net>,<p>I know that IList is the interface and List...,2013-10-30T13:47:05.220,When to use IList and when to use List,123730
2,2008-08-29T18:48:17.390,0,2582.0,35007,38410.0,2582.0,30.0,2008-09-16T15:05:40.703,45,7,<c#><.net><architecture>,<p>Every time I create an object that has a co...,2015-10-02T19:57:45.717,How to expose a collection property?,19805
3,2008-09-02T13:05:48.630,0,12870.0,39447,39467.0,3205.0,4.0,2008-09-24T15:26:01.903,10,6,<c#><windows-mobile><compact-framework><.net-2.0>,<p>I have a class property exposing an interna...,2010-09-13T01:34:36.987,How can I expose only a fragment of IList<>?,1444
4,2008-09-04T03:08:51.760,0,20310.0,43126,43141.0,4140.0,1.0,2008-10-09T21:58:48.070,4,11,<c#><.net><nhibernate><.net-3.5><syntax>,<pre><code>public static IList&lt;T&gt; LoadOb...,2010-10-18T12:50:20.263,"IList.Cast<typeof(T)>() returns error, syntax ...",4759
5,2008-09-04T11:17:57.747,2,107625.0,43500,43505.0,2348.0,16.0,2018-06-09T14:11:56.930,162,14,<c#><.net><collections>,<p>I would like to compare the contents of a c...,2018-08-20T02:01:59.997,Is there a built-in method to compare collecti...,69767
6,2008-09-08T05:51:35.483,0,3817004.0,49220,49221.0,NaN,1.0,2017-12-14T10:06:03.960,2,1,<nhibernate>,<p>I've got two tables in my database: Article...,2017-12-14T10:06:03.960,How can I map a list of strings to my entity u...,2424
7,2008-09-11T16:15:52.587,1,-1.0,57020,NaN,1695.0,7.0,2017-05-23T12:01:23.407,34,11,<c#><.net-3.5><collections>,<p>Was considering the <code>System.Collection...,2012-01-12T15:30:13.313,Which .Net collection for adding multiple obje...,18730
8,2008-09-11T16:36:20.497,1,3440302.0,57054,57166.0,1228.0,1.0,2014-04-23T06:17:40.727,7,5,<.net><generics><interface><explicit>,<p>I've got a collection that implements an in...,2014-04-23T06:17:40.727,How to solve call ambiguity between Generic.IL...,2417
9,2008-09-15T04:07:11.370,1,4140.0,61850,61854.0,4140.0,NaN,2008-09-15T05:01:12.770,3,3,<c#><javascript><ajax><.net-3.5><repeater>,"<p>Ok, so there has to be a way to do this... ...",2008-11-08T05:31:05.327,Is there a way to asynchronously filter an IList?,827
